In [ ]:
%env SPARK_HOME /usr/local/lib/python3.6/dist-packages/pyspark

In [ ]:
!pip install pyarrow

In [ ]:
# Data engineering in Dotscience
# ==============================
#
# In this notebook, we take a raw dataset from Databricks,
# split it into training, test and validation sets, and persist
# a snapshot to the Dotscience filesystem for reproducibility.

import dotscience as ds
import numpy as np
import pickle
import os
from pyspark.sql import SparkSession
from pyspark.ml import image

ds.start()

SEED = 42

spark = SparkSession.builder.getOrCreate()
df = spark.table("gtsrb")

if not os.path.exists("../data"):
    os.mkdir("../data")

# Sample the data
train_df, test_df, validate_df = df.randomSplit([0.6, 0.3, 0.1], seed=SEED)

for name, df in (("train", train_df), ("test", test_df), ("validate", validate_df)):
    result = df.toPandas()
    pickle.dump(result, open(ds.output("../data/databricks-%s.p" % (name,)), "wb"))

ds.publish("ingested data from spark")